In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [3]:
device = torch.device ( "cuda:0" if torch.cuda.is_available() else "cpu" )
print ( device )

cuda:0


In [12]:
df = pd.read_csv("../datasets/Petitions.csv")
corpus = df.sample(10000, replace=None, random_state=0)
corpus

,id,public_petition_text,reason_category
10014,3041543,На протяжении нескольких дней из крана идет гр...,Водоснабжение
29860,3296230,граффити на стенде детской площадки,Благоустройство
47582,3322077,По адресу пер. Джамбула д. 12 на фасаде МКД на...,Нарушение правил пользования общим имуществом
27828,3080599,Бумажные объявления на ТСОДД,Благоустройство
10170,3301487,ямы на дороге,Благоустройство
...,...,...,...
36775,3041604,Незаконная торговля не пресечена,Незаконная реализация товаров с торгового обор...
39271,2994297,Надписи на будке.,Благоустройство
44990,3236936,"Снег, лед. Необходимо убрать",Благоустройство
59226,3245246,Влажное подметание лестничных площадок и марше...,Содержание МКД


In [13]:
corpus_text = corpus["public_petition_text"]
corpus_text_category = corpus["reason_category"]

In [14]:
print(corpus_text)

10014    На протяжении нескольких дней из крана идет гр...
29860                  граффити на стенде детской площадки
47582    По адресу пер. Джамбула д. 12 на фасаде МКД на...
27828                         Бумажные объявления на ТСОДД
10170                                        ямы на дороге
                               ...                        
36775                     Незаконная торговля не пресечена
39271                                    Надписи на будке.
44990                         Снег, лед. Необходимо убрать
59226    Влажное подметание лестничных площадок и марше...
25841    Затоплен подвал грязевой жижей, воняет. Парадн...
Name: public_petition_text, Length: 10000, dtype: object


In [25]:
from nltk.corpus import stopwords
def prepare_data(corpus):
    stemmer = SnowballStemmer('russian')
    from nltk.corpus import stopwords
    stopwords = stopwords.words('russian')
    prepared_data = []
    for petition in corpus:   
        data = petition.lower()
        data1 = re.sub('[\\r|\\n]+', ' ', data)
        data2 = re.sub('[a-zA-Z]+', '', data1)
        data3 = re.sub('[0-9]+', '', data2)
        data4 = re.sub('[^\s^\w]+', '', data3)
        data5 = word_tokenize(data4)
        data6 = [stemmer.stem(w) for w in data5]
        data7 = [w for w in data6 if w not in stopwords]
        prepared_data.append(data7)
    return prepared_data

In [26]:
corpus_text_prep = prepare_data(corpus_text)

In [30]:
from sklearn.preprocessing import LabelEncoder

In [38]:
le = LabelEncoder()
corpus_text_category_encooded = le.fit_transform(corpus_text_category)
dict(zip(range(0, len(set(corpus_text_category_encooded))),le.inverse_transform(list(set(corpus_text_category_encooded)))))

{0: 'Благоустройство',
 1: 'Водоотведение',
 2: 'Водоснабжение',
 3: 'Кровля',
 4: 'Нарушение порядка пользования общим имуществом',
 5: 'Нарушение правил пользования общим имуществом',
 6: 'Незаконная информационная и (или) рекламная конструкция',
 7: 'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)',
 8: 'Повреждения или неисправность элементов уличной инфраструктуры',
 9: 'Подвалы',
 10: 'Санитарное состояние',
 11: 'Содержание МКД',
 12: 'Состояние рекламных или информационных конструкций',
 13: 'Фасад',
 14: 'Центральное отопление'}

In [15]:
ds = TensorDataset(torch.tensor([tupl[0] for tupl in new_data]), torch.tensor([tupl[1] for tupl in new_data]))
dl = DataLoader(ds, batch_size=32, shuffle=True)

In [16]:
class CBoW(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBoW, self).__init__()
        self.embedder = nn.Embedding(vocab_size, 40)
        self.first_linear = nn.Linear(embed_dim, 300)
        self.second_linear = nn.Linear(300, vocab_size)
    def forward(self, x):
        y = self.embedder(x).mean(dim=1)
        y = self.first_linear(y)
        y = self.second_linear(y)
        return y

In [17]:
loss = nn.CrossEntropyLoss()
model = CBoW(len(vocab), 40)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [20]:
epochs = 5
for epoch in range(epochs):
    batch = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer.zero_grad()
        batch+=1
        outputs = model(x_b)
        loss_value = loss(outputs, y_b)
        loss_value.backward()
        
        optimizer.step()
        
        sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 5.364560127258301


Эпоха 2, Значение функции потерь: 4.3717193603515625


Эпоха 3, Значение функции потерь: 5.197719097137451


Эпоха 4, Значение функции потерь: 5.836966037750244


Эпоха 5, Значение функции потерь: 4.844501495361328


In [33]:
model.to("cpu")
cbow_mt = []
for i in vocab:
    cbow_mt.append(model.first_linear(model.embedder(torch.tensor(metadata[i]))).detach().numpy())

In [36]:
cbow_mt1 = pd.DataFrame(cbow_mt)

In [37]:
with open('cbow_mt.tsv','w', encoding="utf-8") as write_tsv:
    write_tsv.write(cbow_mt1.to_csv(sep='\t', index=False))